In [ ]:
from pyspark.sql import SparkSession
import os
import boto3

spark = SparkSession.builder.appName("read_csv_from_s3").getOrCreate()

s3_bucket_name = "projeto-puc"
s3_folder_path = "raw_db/"

# Listar todos os arquivos do S3
s3_files = []
for s3_obj in boto3.resource('s3').Bucket(s3_bucket_name).objects.filter(Prefix=s3_folder_path):
    if s3_obj.key.endswith('.csv'):
        s3_files.append("s3://" + s3_bucket_name + "/" + s3_obj.key)

for s3_file in s3_files:
    df = spark.read.csv(s3_file, header=True, inferSchema=True, sep=";")

    # Colunas incomuns que deverão ser excluídas
    cols_to_drop = ['Gestor','Canal de Origem','Ano Abertura','Mês Abertura','Data Abertura','Hora Abertura',	'Data Resposta','Hora Resposta','Data Análise','Hora Análise','Data Recusa','Hora Recusa','Hora Finalização','Prazo Resposta','Prazo Analise Gestor','Contratou','Nota do Consumidor','Análise da Recusa','Edição de Conteúdo','Interação do Gestor','Total']

    # Verifica se existe colunas em todos arquivos e remove as colunas listadas
    if cols_to_drop:    
        for col in cols_to_drop:
            df = df.drop(col)

    # Remove linhas duplicadas
    df = df.dropDuplicates()

    # Remover linhas em branco
    df = df.dropna()
    
    # Após as transformações salva os arquivos na pasta ref_db no S3
    output_filename = s3_file.split("/")[-1]  
    df.write.mode("overwrite").csv("s3://" + s3_bucket_name + "/ref_db/" + output_filename, header=True, sep=";")
